<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

<a name='s1-2.2'></a>
### Memory Utilization ###
Memory utilization on a DataFrame depends largely on the date types for each column. 

<p><img src='images/dtypes.png' width=720></p>

We can use `DataFrame.memory_usage()` to see the memory usage for each column (in bytes). Most of the common data types have a fixed size in memory, such as `int`, `float`, `datetime`, and `bool`. Memory usage for these data types is the respective memory requirement multiplied by the number of data points. For `string` data types, the memory usage reported is the number of data points times 8 bytes. This accounts for the 64-bit required for the pointer that points to an address in memory but doesn't include the memory used for the actual string values. The actual memory required for a `string` value is 49 bytes plus an additional byte for each character. The `deep` parameter provides a more accurate memory usage report that accounts for the system-level memory consumption of the contained `string` data type. 

Separately, we've provided a `dli_utils.make_decimal()` function to convert memory size into units based on powers of 2. In contrast to units based on powers of 10, this customary convention is commonly used to report memory capacity. More information about the two definitions can be found [here](https://en.wikipedia.org/wiki/Byte#Multiple-byte_units). 

In [1]:
# import dependencies
import pandas as pd
import sys
import random

# import utility
from dli_utils import make_decimal

# import data
df=pd.read_csv('2020-Mar.csv')

# preview DataFrame
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-03-01 00:00:00 UTC,view,10900348,2232732105912091273,appliances.kitchen.mixer,kitfort,117.63,550295099,23f5ab9e-7774-416d-900b-f2725da671aa
1,2020-03-01 00:00:01 UTC,view,14300020,2232732107967300275,apparel.underwear,casio,264.37,622070532,edf19213-f0dd-46e1-ae18-28968dbddd6e
2,2020-03-01 00:00:03 UTC,view,13200063,2232732061804790604,furniture.bedroom.bed,NaN,126.10,552832344,608087d8-6b69-21db-50a3-2b91ba015532
3,2020-03-01 00:00:03 UTC,view,100077498,2232732085846540487,construction.tools.generator,tamina,424.72,613391860,5ab6c3d5-edd7-4e15-8b0a-75b467a37fd0
4,2020-03-01 00:00:04 UTC,view,1005014,2232732093077520756,construction.tools.light,samsung,482.73,517021211,0c34308d-c455-40bb-9992-3e44920bc2b9


In [2]:
# convert feature as datetime data type
df['event_time']=pd.to_datetime(df['event_time'])

In [3]:
# lists each column at 8 bytes/row
memory_usage_df=df.memory_usage(index=False)
memory_usage_df.name='memory_usage'
dtypes_df=df.dtypes
dtypes_df.name='dtype'

# show each column uses roughly number of rows * 8 bytes
# 8 bytes from 64-bit numerical data as well as 8 bytes to store a pointer for object data type
byte_size=len(df) * 8 * len(df.columns)

print(f'Total memory use is {byte_size} bytes or ~{make_decimal(byte_size)}.')

pd.concat([memory_usage_df, dtypes_df], axis=1)

Total memory use is 4056569352 bytes or ~3.78 GB.


,memory_usage,dtype
event_time,450729928,"datetime64[ns, UTC]"
event_type,450729928,object
product_id,450729928,int64
category_id,450729928,int64
category_code,450729928,object
brand,450729928,object
price,450729928,float64
user_id,450729928,int64
user_session,450729928,object


In [4]:
# lists each column's full memory usage
memory_usage_df=df.memory_usage(deep=True, index=False)
memory_usage_df.name='memory_usage'

byte_size=memory_usage_df.sum()

# show total memory usage
print(f'Total memory use is {byte_size} bytes or ~{make_decimal(byte_size)}.')

pd.concat([memory_usage_df, dtypes_df], axis=1)

Total memory use is 18435137127 bytes or ~17.17 GB.


,memory_usage,dtype
event_time,450729928,"datetime64[ns, UTC]"
event_type,3440915437,object
product_id,450729928,int64
category_id,450729928,int64
category_code,4208566142,object
brand,3292273606,object
price,450729928,float64
user_id,450729928,int64
user_session,5239732302,object


In [5]:
# alternatively, use sys.getsizeof() instead
byte_size=sys.getsizeof(df)

print(f'Total memory use is {byte_size} bytes or ~{make_decimal(byte_size)}.')

Total memory use is 18435137271 bytes or ~17.17 GB.


In [6]:
# check random string-typed column
string_cols=[col for col in df.columns if df[col].dtype=='object' ]
column_to_check=random.choice(string_cols)

overhead=49
pointer_size=8

# nan==nan when value is not a number
# nan uses 32 bytes of memory
print(f'{column_to_check} column uses : {sum([(len(item)+overhead+pointer_size) if item==item else 32 for item in df[column_to_check].values])} bytes of memory.')

event_type column uses : 3440915437 bytes of memory.


<p><img src='images/tip.png' width=720></p>
When Python stores a string, it actually uses memory for the overhead of the Python object, metadata about the string, and the string itself. The amount of memory usage we calculated includes temporary objects that get deallocated after the initial import. It's important to note that Python has memory optimization mechanics for strings such that when the same string is created multiple time, Python will cache or "intern" it in memory and reuse it for later string objects. 

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>